In [1]:
import pandas as pd
import os

In [2]:
os.chdir("/home/573/rh2942/WASSA-2023-EMP") # changing dir for evaluation file
# print(os.getcwd())

# Prediction

In [ ]:
from main_methods import final_prediction

In [3]:
from main_methods import train_test_wo_acc

In [4]:
task='personality_conscientiousness'
# final_prediction(task=task, lr=5.98e-05, batch=7, seed=30)
train_test_wo_acc(task, 5.98e-05, 7, 30)
# predictions_cons = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

personality_conscientiousness prediction


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Map:   0%|          | 0/1558 [00:00<?, ? examples/s]

Map:   0%|          | 0/208 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 1: average loss = 2.8493718575402225
0.4413
Epoch 2: average loss = 0.23210113785217093
0.6359
Epoch 3: average loss = 0.07626993111084161
0.7232
Epoch 4: average loss = 0.04252516280309216
0.7261
Epoch 5: average loss = 0.018529046152644234
0.7448
Epoch 6: average loss = 0.007226667880480524
0.7662
Epoch 7: average loss = 0.0035103787983884627
0.751
Epoch 8: average loss = 0.002298602912748063
0.7496
Epoch 9: average loss = 0.0014946443289638425
0.7545
Epoch 10: average loss = 0.0009497533146265347
0.7507
Epoch 11: average loss = 0.0007395622734240264
0.7424
Epoch 12: average loss = 0.0006054609859078515
0.7477
Epoch 13: average loss = 0.0004896419810605207
0.743
Epoch 14: average loss = 0.00040189739879903423
0.7515
Epoch 15: average loss = 0.0003607446178566819
0.7481
Epoch 16: average loss = 0.0006936200521204432
0.7523
Epoch 17: average loss = 0.0008382561364616508
0.7485
Epoch 18: average loss = 0.0008072679537865793
0.7468
Epoch 19: average loss = 0.0005334740193322407
0.7

In [ ]:
task='personality_openess'
final_prediction(task=task, lr=1.80e-05, batch=2, seed=81)
predictions_open = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

In [ ]:
task='personality_extraversion'
final_prediction(task=task, lr=1.61e-05, batch=7, seed=34)
predictions_extr = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

In [ ]:
task='personality_agreeableness'
final_prediction(task=task, lr=5.15e-05, batch=6, seed=65)
predictions_agre = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

In [ ]:
task='personality_stability'
final_prediction(task=task, lr=5.36e-05, batch=5, seed=13)
predictions_stab = pd.read_csv("./prediction/predictions_" + task + ".tsv", sep='\t', header=None)

In [ ]:
predictions_PER = pd.concat([predictions_cons, predictions_open, predictions_extr, predictions_agre, predictions_stab], axis=1)

predictions_PER.to_csv("./prediction/predictions_PER.tsv", sep='\t', header=False, index=False)

In [ ]:
from evaluation import pearsonr, calculate_pearson

In [ ]:
# # Just checking the dev set performance
gold_dev = pd.read_csv('./dataset/dev/goldstandard_dev.tsv', sep='\t', header=None) # no header
predictions_PER= pd.read_csv("./prediction/predictions_PER.tsv", sep='\t', header=None)

print(f"cons: {pearsonr(gold_dev.loc[:,3].tolist(), predictions_PER.loc[:,0].tolist())}")
print(f"open: {pearsonr(gold_dev.loc[:,4].tolist(), predictions_PER.loc[:,1].tolist())}")
print(f"extr: {pearsonr(gold_dev.loc[:,5].tolist(), predictions_PER.loc[:,2].tolist())}")
print(f"agre: {pearsonr(gold_dev.loc[:,6].tolist(), predictions_PER.loc[:,3].tolist())}")
print(f"stab: {pearsonr(gold_dev.loc[:,7].tolist(), predictions_PER.loc[:,4].tolist())}")

# Hyperparam tuning

In [ ]:
from hyperparam_tuner import optuna_tuner

In [ ]:
tasks = [
    'personality_conscientiousness',
    'personality_openess',
    'personality_extraversion',
    'personality_agreeableness',
    'personality_stability'
]

for task in tasks:
    optuna_tuner(task=task)